In [1]:
from collections import OrderedDict
import torch
from torch import nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from base import BaseModel
import numpy as np
from xml.etree import ElementTree
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()

    def forward(self, x):
        N = x.data.size(0)
        C = x.data.size(1)
        H = x.data.size(2)
        W = x.data.size(3)
        x = F.avg_pool2d(x, (H, W))
        x = x.view(N, C)
        return x

In [3]:
class Darknet19(BaseModel):
    def __init__(self, pretrained=True):
        super(Darknet19, self).__init__()

        self.features = nn.Sequential(OrderedDict([
        ('layer1', nn.Sequential(OrderedDict([
            ('conv1_1', nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn1_1', nn.BatchNorm2d(32)),
            ('leaky1_1', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool1', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer2', nn.Sequential(OrderedDict([
            ('conv2_1', nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn2_1', nn.BatchNorm2d(64)),
            ('leaky2_1', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool2', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer3', nn.Sequential(OrderedDict([
            ('conv3_1', nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn3_1', nn.BatchNorm2d(128)),
            ('leaky3_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv3_2', nn.Conv2d(128, 64, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn3_2', nn.BatchNorm2d(64)),
            ('leaky3_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv3_3', nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn3_3', nn.BatchNorm2d(128)),
            ('leaky3_3', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool3', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer4', nn.Sequential(OrderedDict([
            ('conv4_1', nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn4_1', nn.BatchNorm2d(256)),
            ('leaky4_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv4_2', nn.Conv2d(256, 128, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn4_2', nn.BatchNorm2d(128)),
            ('leaky4_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv4_3', nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn4_3', nn.BatchNorm2d(256)),
            ('leaky4_3', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool4', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer5', nn.Sequential(OrderedDict([
            ('conv5_1', nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn5_1', nn.BatchNorm2d(512)),
            ('leaky5_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_2', nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn5_2', nn.BatchNorm2d(256)),
            ('leaky5_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_3', nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn5_3', nn.BatchNorm2d(512)),
            ('leaky5_3', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_4', nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=1, bias=False)),
            ('bn5_4', nn.BatchNorm2d(256)),
            ('leaky5_4', nn.LeakyReLU(0.1, inplace=True)),
            ('conv5_5', nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn5_5', nn.BatchNorm2d(512)),
            ('leaky5_5', nn.LeakyReLU(0.1, inplace=True)),
            ('maxpool5', nn.MaxPool2d(kernel_size=2, stride=2))
        ]))),
        ('layer6', nn.Sequential(OrderedDict([
            ('conv6_1', nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn6_1', nn.BatchNorm2d(1024)),
            ('leaky6_1', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_2', nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=0, bias=False)),
            ('bn6_2', nn.BatchNorm2d(512)),
            ('leaky6_2', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_3', nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn6_3', nn.BatchNorm2d(1024)),
            ('leaky6_3', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_4', nn.Conv2d(1024, 512, kernel_size=1, stride=1, padding=1, bias=False)),
            ('bn6_4', nn.BatchNorm2d(512)),
            ('leaky6_4', nn.LeakyReLU(0.1, inplace=True)),
            ('conv6_5', nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1, bias=False)),
            ('bn6_5', nn.BatchNorm2d(1024)),
            ('leaky6_5', nn.LeakyReLU(0.1, inplace=True))
        ])))
        ]))

        self.classifier = nn.Sequential(OrderedDict([
        ('classifier', nn.Sequential(OrderedDict([
        ('conv7_1', nn.Conv2d(1024, 1000, kernel_size=(1, 1), stride=(1, 1))),
        ('globalavgpool', GlobalAvgPool2d()),
        ('softmax', nn.Softmax(dim=1))
        ])))]))

        if pretrained:
            # self.load_state_dict(model_zoo.load_url(model_paths['darknet19'],  progress=True))
            self.load_weight()
            print('Model is loaded')

    def forward(self, x):
        out = self.features(x)
        out = self.classifier(out)
        return out
    def load_weight(self):
        weight_file = 'weights/darknet19-deepBakSu-e1b3ec1e.pth'
        assert len(torch.load(weight_file).keys()) == len(self.state_dict().keys())
        dic = {}
        for now_keys, values in zip(self.state_dict().keys(), torch.load(weight_file).values()):
            dic[now_keys]=values
        self.load_state_dict(dic)
        print('Weights are loaded!')

In [4]:
model = Darknet19(True)
model = model.eval()

Weights are loaded!
Model is loaded


In [5]:
input_tensor = torch.FloatTensor(np.array([1.0] * 224 * 224 * 3))
input_tensor = input_tensor.reshape([1, 3, 224, 224])

In [6]:
idx = 0
def get_output(layer, eval = True) :
    if eval:
        layer = layer.eval()
    global input_tensor, idx
    input_tensor = layer(input_tensor)
    idx += 1
    print("Output of layer ", idx, " : ", input_tensor.sum())

In [7]:
def reset():
    global input_tensor, idx
    input_tensor = torch.FloatTensor(np.array([1.0] * 224 * 224 * 3))
    input_tensor = input_tensor.reshape([1, 3, 224, 224])
    idx = 0

In [8]:
model = model.eval()
reset()
for layer in model.features[0]:
    get_output(layer)

Output of layer  1  :  tensor(-39838.8086, grad_fn=<SumBackward0>)
Output of layer  2  :  tensor(-3260192., grad_fn=<SumBackward0>)
Output of layer  3  :  tensor(816813.1250, grad_fn=<SumBackward0>)
Output of layer  4  :  tensor(216611.4688, grad_fn=<SumBackward0>)


In [9]:
model = model.eval()
reset()
for layer in model.features[0]:
    get_output(layer)

Output of layer  1  :  tensor(-39838.8086, grad_fn=<SumBackward0>)
Output of layer  2  :  tensor(-3260192., grad_fn=<SumBackward0>)
Output of layer  3  :  tensor(816813.1250, grad_fn=<SumBackward0>)
Output of layer  4  :  tensor(216611.4688, grad_fn=<SumBackward0>)


In [10]:
model = model.eval()
for layer in model.features[1]:
    get_output(layer)

Output of layer  5  :  tensor(-248550.6250, grad_fn=<SumBackward0>)
Output of layer  6  :  tensor(147896.6094, grad_fn=<SumBackward0>)
Output of layer  7  :  tensor(619941., grad_fn=<SumBackward0>)
Output of layer  8  :  tensor(164179.2031, grad_fn=<SumBackward0>)


In [11]:
model = model.eval()
for layer in model.features[2]:
    get_output(layer)

Output of layer  9  :  tensor(-146027.7344, grad_fn=<SumBackward0>)
Output of layer  10  :  tensor(88665.1016, grad_fn=<SumBackward0>)
Output of layer  11  :  tensor(255238.5312, grad_fn=<SumBackward0>)
Output of layer  12  :  tensor(-30494.3613, grad_fn=<SumBackward0>)
Output of layer  13  :  tensor(46894.4727, grad_fn=<SumBackward0>)
Output of layer  14  :  tensor(187059.7500, grad_fn=<SumBackward0>)
Output of layer  15  :  tensor(-133359.8750, grad_fn=<SumBackward0>)
Output of layer  16  :  tensor(-305293.2188, grad_fn=<SumBackward0>)
Output of layer  17  :  tensor(23133.1074, grad_fn=<SumBackward0>)
Output of layer  18  :  tensor(8483.1250, grad_fn=<SumBackward0>)


In [12]:
model = model.eval()
for layer in model.features[3]:
    get_output(layer)

Output of layer  19  :  tensor(-25217., grad_fn=<SumBackward0>)
Output of layer  20  :  tensor(-90406.0938, grad_fn=<SumBackward0>)
Output of layer  21  :  tensor(53217.3828, grad_fn=<SumBackward0>)
Output of layer  22  :  tensor(-11441.3535, grad_fn=<SumBackward0>)
Output of layer  23  :  tensor(-10056.9932, grad_fn=<SumBackward0>)
Output of layer  24  :  tensor(54672.6250, grad_fn=<SumBackward0>)
Output of layer  25  :  tensor(-181565.1250, grad_fn=<SumBackward0>)
Output of layer  26  :  tensor(-399055.4688, grad_fn=<SumBackward0>)
Output of layer  27  :  tensor(-30219.5000, grad_fn=<SumBackward0>)
Output of layer  28  :  tensor(-6255.7974, grad_fn=<SumBackward0>)


In [13]:
model = model.eval()
for layer in model.features[4]:
    get_output(layer)

Output of layer  29  :  tensor(-8978.9277, grad_fn=<SumBackward0>)
Output of layer  30  :  tensor(-91405.1172, grad_fn=<SumBackward0>)
Output of layer  31  :  tensor(924.0886, grad_fn=<SumBackward0>)
Output of layer  32  :  tensor(-435.5790, grad_fn=<SumBackward0>)
Output of layer  33  :  tensor(-22883.8809, grad_fn=<SumBackward0>)
Output of layer  34  :  tensor(10422.3750, grad_fn=<SumBackward0>)
Output of layer  35  :  tensor(-18061.9473, grad_fn=<SumBackward0>)
Output of layer  36  :  tensor(-31303.4258, grad_fn=<SumBackward0>)
Output of layer  37  :  tensor(20105.5391, grad_fn=<SumBackward0>)
Output of layer  38  :  tensor(-11858.4180, grad_fn=<SumBackward0>)
Output of layer  39  :  tensor(-21867.9277, grad_fn=<SumBackward0>)
Output of layer  40  :  tensor(17077.5664, grad_fn=<SumBackward0>)
Output of layer  41  :  tensor(-34578.3711, grad_fn=<SumBackward0>)
Output of layer  42  :  tensor(-185575.1562, grad_fn=<SumBackward0>)
Output of layer  43  :  tensor(-14384.5020, grad_fn=<Sum

In [14]:
model = model.eval()
for layer in model.features[5]:
    get_output(layer)

Output of layer  45  :  tensor(-1740.2012, grad_fn=<SumBackward0>)
Output of layer  46  :  tensor(-57422.6250, grad_fn=<SumBackward0>)
Output of layer  47  :  tensor(-3817.9036, grad_fn=<SumBackward0>)
Output of layer  48  :  tensor(3776.1975, grad_fn=<SumBackward0>)
Output of layer  49  :  tensor(-17405.7832, grad_fn=<SumBackward0>)
Output of layer  50  :  tensor(784.8477, grad_fn=<SumBackward0>)
Output of layer  51  :  tensor(-5934.9399, grad_fn=<SumBackward0>)
Output of layer  52  :  tensor(-51272.7969, grad_fn=<SumBackward0>)
Output of layer  53  :  tensor(-3034.4202, grad_fn=<SumBackward0>)
Output of layer  54  :  tensor(3262.7397, grad_fn=<SumBackward0>)
Output of layer  55  :  tensor(-43840.2773, grad_fn=<SumBackward0>)
Output of layer  56  :  tensor(-523.1497, grad_fn=<SumBackward0>)
Output of layer  57  :  tensor(206.3079, grad_fn=<SumBackward0>)
Output of layer  58  :  tensor(-298738.4062, grad_fn=<SumBackward0>)
Output of layer  59  :  tensor(-491.2022, grad_fn=<SumBackward0

In [15]:
model = model.eval()
for layer in model.classifier[0]:
    get_output(layer)

Output of layer  60  :  tensor(0.3704, grad_fn=<SumBackward0>)
Output of layer  61  :  tensor(0.0037, grad_fn=<SumBackward0>)
Output of layer  62  :  tensor(1.0000, grad_fn=<SumBackward0>)


In [16]:
input_tensor.argmax()

tensor(521)

In [17]:
reset()
model = model.eval()
model(input_tensor).sum()
torch.argmax(model(input_tensor))

tensor(521)

In [24]:
from torchvision import transforms
mat = []

In [29]:
from torchvision import transforms
from PIL import Image 
model.eval()
classes = os.listdir("../imagenette-test")
for folder in classes :
    print(folder)
    images = os.listdir("../imagenette-test/" + folder)
    for image in images:
        train_transforms = transforms.Compose([transforms.Resize(224), transforms.ToTensor()])
        img = Image.open("../imagenette-test/" + folder + "/" + image)
        img = train_transforms(img)
        temp = img
        img = img.unsqueeze(0)

        index = torch.argmax(model(img))
        print(index, " ---->", model(img)[0][index], " ---> ", image, "------>" ,img.sum())
        reverse_transform = transforms.ToPILImage()
        temp = reverse_transform(temp)
        base_dest_path = "../imagenette_new/"
        if not os.path.exists(base_dest_path) :
            os.mkdir(base_dest_path)
        dest_folder = folder
        if not os.path.exists(base_dest_path + folder) :
            os.mkdir(base_dest_path + folder)
        image_path = base_dest_path + folder + "/" + image
        temp.save(image_path)
        mat.append(img.numpy().ravel())

n03394916
tensor(347)  ----> tensor(0.0746, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00003759.jpg ------> tensor(68339.8203)
tensor(347)  ----> tensor(0.0765, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00005548.jpg ------> tensor(25738.9199)
tensor(347)  ----> tensor(0.2213, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00007985.jpg ------> tensor(46697.1094)
tensor(346)  ----> tensor(0.8670, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00005554.jpg ------> tensor(19992.7109)
tensor(347)  ----> tensor(0.5003, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00007155.jpg ------> tensor(25416.3613)
tensor(346)  ----> tensor(0.1369, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00000957.jpg ------> tensor(25532.3027)
tensor(348)  ----> tensor(0.1855, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00003587.jpg ------> tensor(68653.9297)
tensor(347)  ----> tensor(0.0933, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00007536.jpg ------> tensor(34702.4531)
n03417042
tens

In [27]:
np.savetxt("../imagenette_image.csv", mat, delimiter=',')

In [30]:
test_image = Image.open("../imagenette_new/n03394916/ILSVRC2012_val_00003759.jpg")

In [32]:
test_image = test_image.resize((224, 224))
test_image_matrix = np.asarray(test_image, dtype = np.uint8)
test_image_matrix = test_image_matrix / 255
test_image_matrix.sum()

68326.38823529413

In [22]:
(test_tensor.float().div(255)).sum()

tensor(75931.4141)

In [27]:
(np.asarray((test_image)) / 255).sum()

75931.41960784313

In [28]:
transforms.ToTensor()(test_image).sum()

tensor(75931.4375)

In [136]:
from torchvision import transforms
from PIL import Image 
model.eval()
classes = os.listdir("../imagenette-test")
for folder in classes :
    print(folder)
    images = os.listdir("../imagenette-test/" + folder)
    for image in images:
        train_transforms = transforms.Compose([transforms.Resize(224)])
        img = Image.open("../imagenette-test/" + folder + "/" + image)
        img = train_transforms(img)
        base_dest_path = "../imagenette_new/"
        if not os.path.exists(base_dest_path) :
            os.mkdir(base_dest_path)
        dest_folder = folder
        if not os.path.exists(base_dest_path + folder) :
            os.mkdir(base_dest_path + folder)
        image_path = base_dest_path + folder + "/" + image[:-4] + ".png"
        img.save(image_path, "PNG")

n03394916
n03417042
n03445777
n02102040
n03425413
n03888257
n03028079
n03000684
n01440764
n02979186


In [38]:
test_image_matrix.shape

(320, 320, 3)

In [142]:
from torchvision import transforms
from PIL import Image 
model.eval()
classes = os.listdir("../imagenette_new")
for folder in classes :
    print(folder)
    images = os.listdir("../imagenette_new/" + folder)
    for image in images:
        train_transforms = transforms.Compose([transforms.Resize(224), transforms.ToTensor()])
        img = Image.open("../imagenette_new/" + folder + "/" + image)
        img = train_transforms(img)
        temp = img
        img = img.unsqueeze(0)

        index = torch.argmax(model(img))
        print(index, " ---->", model(img)[0][index], " ---> ", image, "------>" ,img.sum())
        reverse_transform = transforms.ToPILImage()
        temp = reverse_transform(temp)
        base_dest_path = "../imagenette_new/"
        if not os.path.exists(base_dest_path) :
            os.mkdir(base_dest_path)
        dest_folder = folder
        if not os.path.exists(base_dest_path + folder) :
            os.mkdir(base_dest_path + folder)
        image_path = base_dest_path + folder + "/" + image
        temp.save(image_path)
        mat.append(img.numpy().ravel())

n03394916
tensor(921)  ----> tensor(0.0718, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00003759.jpg ------> tensor(68326.4688)
tensor(347)  ----> tensor(0.0746, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00003759.png ------> tensor(68339.8203)
tensor(347)  ----> tensor(0.0765, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00005548.png ------> tensor(25738.9199)
tensor(347)  ----> tensor(0.0610, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00005548.jpg ------> tensor(25834.2227)
tensor(347)  ----> tensor(0.2213, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00007985.png ------> tensor(46697.1094)
tensor(347)  ----> tensor(0.1094, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00007985.jpg ------> tensor(46698.8281)
tensor(346)  ----> tensor(0.8628, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00005554.jpg ------> tensor(20014.7852)
tensor(347)  ----> tensor(0.5003, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00007155.png ------> tensor(25416.3613)
tensor(347)  -

In [57]:
from torchvision import transforms
from PIL import Image 
model.eval()
classes = os.listdir("../imagenette-test")
for folder in classes :
    print(folder)
    images = os.listdir("../imagenette_new/" + folder)
    for image in images:
        if image[-4:] != ".png" :
            continue
        train_transforms = transforms.Compose([transforms.ToTensor()])
        img = Image.open("../imagenette_new/" + folder + "/" + image)
        img_np = np.asanyarray(img, dtype = np.uint8)
        img_np = (img_np) / 255.0
        img = train_transforms(img_np)
        img = img.unsqueeze(0)
        img = img.float()
        index = torch.argmax(model(img))
        print(index, " ---->", model(img)[0][index], " ---> ", image, "------>" ,img.sum())

n03394916
tensor(347)  ----> tensor(0.0746, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00003759.png ------> tensor(68339.7812)
tensor(347)  ----> tensor(0.0765, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00005548.png ------> tensor(25738.6816)
tensor(347)  ----> tensor(0.2213, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00007985.png ------> tensor(46697.1055)
tensor(347)  ----> tensor(0.5003, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00007155.png ------> tensor(25416.3379)
tensor(346)  ----> tensor(0.8670, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00005554.png ------> tensor(19992.6191)
tensor(346)  ----> tensor(0.1369, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00000957.png ------> tensor(25532.0078)
tensor(348)  ----> tensor(0.1855, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00003587.png ------> tensor(68653.9141)
tensor(347)  ----> tensor(0.0933, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00007536.png ------> tensor(34702.4453)
n03417042
tens

In [56]:
from torchvision import transforms
from PIL import Image 
model.eval()
classes = os.listdir("../imagenette-test")
for folder in classes :
    print(folder)
    images = os.listdir("../imagenette-test/" + folder)
    for image in images:
        train_transforms = transforms.Compose([transforms.Resize(224), transforms.ToTensor()])
        img = Image.open("../imagenette-test/" + folder + "/" + image)
        img = train_transforms(img)
        img = img.unsqueeze(0)
        img = img.float()
        index = torch.argmax(model(img))
        print(index, " ---->", model(img)[0][index], " ---> ", image, "------>" ,img.sum())

n03394916
tensor(347)  ----> tensor(0.0746, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00003759.jpg ------> tensor(68339.8203)
tensor(347)  ----> tensor(0.0765, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00005548.jpg ------> tensor(25738.9199)
tensor(347)  ----> tensor(0.2213, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00007985.jpg ------> tensor(46697.1094)
tensor(346)  ----> tensor(0.8670, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00005554.jpg ------> tensor(19992.7109)
tensor(347)  ----> tensor(0.5003, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00007155.jpg ------> tensor(25416.3613)
tensor(346)  ----> tensor(0.1369, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00000957.jpg ------> tensor(25532.3027)
tensor(348)  ----> tensor(0.1855, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00003587.jpg ------> tensor(68653.9297)
tensor(347)  ----> tensor(0.0933, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00007536.jpg ------> tensor(34702.4531)
n03417042
tens